# Machine Learning in Jupyter Notebooks

## Part 0 - Setup environment and download sample data

In [4]:
# Environment setup
from pathlib import Path
from IPython.display import display, Markdown, HTML
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

display(HTML("<h3>Starting Notebook setup ...</h3>"))

import msticpy as mp
mp.init_notebook(namespace=globals())

In [5]:
# Retrieve sample data files
from urllib.request import urlretrieve  # download files from URLs to local
from pathlib import Path    # modern way to handle file system paths
from tqdm.auto import tqdm  # provide progress bars for the download loop

github_uri = "https://raw.githubusercontent.com/Azure/Azure-Sentinel-Notebooks/master/{file_name}"
github_files = {
    "exchange_admin.pkl": "src/data",
    "processes_on_host.pkl": "src/data",
    "timeseries.pkl": "src/data",
    "data_queries.yaml": "src/data",
}

Path("./src/data").mkdir(exist_ok=True, parents=True)
for file, path in tqdm(github_files.items(), desc="File download"):
    file_path = Path(path).joinpath(file)
    url_path = f"{path}/{file}" if path else file
    urlretrieve(
        github_uri.format(file_name=url_path),
        file_path
    )
    assert Path(file_path).is_file()

File download:   0%|          | 0/4 [00:00<?, ?it/s]

## Part 1 - Time Series Analysis

In [6]:
from msticpy.nbtools import nbwidgets, nbdisplay

query_range = nbwidgets.QueryTime(
    origin_time=pd.Timestamp("2020-07-13 00:00:00"),
    before=1,
    units="week"
)
query_range

In [7]:
# initialize the data provider and connect to our Splunk instance
qry_prov = QueryProvider("LocalData", data_paths=["./src/data"], query_paths=["./src/data"])
qry_prov.connect()

def md(s): display(Markdown(s))

ob_bytes_per_hour = qry_prov.Network.get_network_summary(query_range)
md("## Sample data:")
ob_bytes_per_hour.head(5)

Connected.


## Sample data:

,TotalBytesSent
TimeGenerated,
2020-07-06 00:00:00+00:00,10823
2020-07-06 01:00:00+00:00,14821
2020-07-06 02:00:00+00:00,13532
2020-07-06 03:00:00+00:00,11947
2020-07-06 04:00:00+00:00,11193


In [8]:
# Detect anomalous network activity
#%pip install msticpy[ml]
from msticpy.analysis.timeseries import display_timeseries_anomalies
from msticpy.analysis.timeseries import timeseries_anomalies_stl

# conduct timeseries analysis
ts_analysis = timeseries_anomalies_stl(ob_bytes_per_hour)

# visualize the timeseries and any anomalies
display_timeseries_anomalies(data = ts_analysis, y = 'TotalBytesSent');

md("## We can see two clearly anomalous data points representing unusual outbound traffic.<hr>")


Loading BokehJS ...

## We can see two clearly anomalous data points representing unusual outbound traffic.<hr>

In [9]:
# view the summary events marked as anomalous
max_score, min_score = ts_analysis.score.max(), ts_analysis.min()
ts_analysis[ts_analysis["anomalies"] == 1]

,TimeGenerated,TotalBytesSent,residual,trend,seasonal,weights,baseline,score,anomalies
114,2020-07-10 18:00:00+00:00,48616,16383,21598,10633,1,32232,6.220873,1
115,2020-07-10 19:00:00+00:00,45856,15949,21373,8532,1,29906,6.055974,1


In [10]:
# extract the anomaly period
# Identify when the anomalies occur so that we can use this timerange to scope the next stage of our investigation.
# Add a 1 hour buffer around the anomalies
import pandas as pd
start = ts_analysis[ts_analysis['anomalies'] == 1]['TimeGenerated'].min() - pd.to_timedelta(1, unit = 'h')
end = ts_analysis[ts_analysis['anomalies'] == 1]['TimeGenerated'].max() + pd.to_timedelta(1, unit = 'h')

md("## Anomalous network traffic detected between:")
md(f"Start time: <b>{start}</b><br>End time: <b>{end}</b><hr>")

## Anomalous network traffic detected between:

Start time: <b>2020-07-10 17:00:00+00:00</b><br>End time: <b>2020-07-10 20:00:00+00:00</b><hr>

# Part 2 - Using Clustering

In [11]:
# aggregating similar process patterns to highlight unusual logon sessions
print("Getting process events...", end="")
processes_on_host = qry_prov.WindowsSecurity.list_host_processes(
    query_range, host_name = "MSTICAlertsWin1"
)

md("## **Initial analysis of data set**")
md(f"Total processes in data set: <b>{len(processes_on_host)}</b>")
for column in ("Account", "NewProcessName", "CommandLine"):
    md(f"Total distinct {column} in data set: <b>{processes_on_host[column].nunique()}</b>")
md("<hr>")
md("## Try grouping by distinct Account, Process, Commandline<br> - we still have 1000s of rows!")
display(
    processes_on_host
    .groupby(["Account", "NewProcessName", "CommandLine"])
    [["TimeGenerated"]]
    .count()
    .rename(columns = {"TimeGenerated": "Count"})
)


Getting process events...

## **Initial analysis of data set**

Total processes in data set: <b>22979</b>

Total distinct Account in data set: <b>5</b>

Total distinct NewProcessName in data set: <b>192</b>

Total distinct CommandLine in data set: <b>4551</b>

<hr>

## Try grouping by distinct Account, Process, Commandline<br> - we still have 1000s of rows!

Count
Account                    NewProcessName                                                                                       CommandLine                                                                                                
MSTICAlertsWin1\MSTICAdmin C:\Program Files (x86)\Internet Explorer\iexplore.exe                                                "C:\Program Files (x86)\Internet Explorer\IEXPLORE.EXE" SCODEF:30680 CREDAT:82945 /prefetch:2             1
                                                                                                                                "C:\Program Files (x86)\Internet Explorer\IEXPLORE.EXE" SCODEF:5820 CREDAT:82945 /prefetch:2              1
                           C:\Program Files\Internet Explorer\iexplore.exe                                                      "C:\Program Files\Internet Explorer\iexplore.exe"                                                         1
                                                                                                                                "C:\Program Files\Internet Explorer\iexplore.exe" -restart /WERRESTART                                    1
                           C:\Program Files\PuTTY\putty.exe                                                                     "C:\Program Files\PuTTY\putty.exe"                                                                        1
...                                                                                                                                                                                                                                     ...
WORKGROUP\MSTICAlertsWin1$ C:\Windows\Temp\CR_42BC8.tmp\setup.exe                                                               C:\Windows\TEMP\CR_42BC8.tmp\setup.exe --type=crashpad-handler /prefetch:7 --monitor-self-annotat...      1
                           C:\Windows\Temp\D398059B-A17E-43B8-95E4-8F0453629D9F\DismHost.exe                                    C:\Windows\TEMP\D398059B-A17E-43B8-95E4-8F0453629D9F\dismhost.exe {5B5DC19A-0D8F-4B1F-8B28-CAE7B1...      1
                           C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.14393.2602_none_7e... C:\Windows\winsxs\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.14393.2602_none_7e...     16
                           C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.14393.2782_none_7e... C:\Windows\winsxs\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.14393.2782_none_7e...     11
                           C:\Windows\servicing\TrustedInstaller.exe                                                            C:\Windows\servicing\TrustedInstaller.exe                                                                26

[4594 rows x 1 columns]

In [12]:
# perform process clustering analysis to identify unusual or rare process execution patterns that might indicate suspicious activity.
from msticpy.sectools.eventcluster import dbcluster_events, add_process_features, char_ord_score
from collections import Counter

print(f"Input data: {len(processes_on_host)} events")
print("Extracting features...", end="")
# feature extraction
feature_procs = add_process_features(input_frame=processes_on_host, path_separator="\\")

feature_procs["accountNum"] = feature_procs.apply(
    lambda x: char_ord_score(x.Account), axis = 1       # convert account names to numerical scores based on character ordinal values, help to compare processes mathematically
)
print(".", end="")

# play around with the max_cluster_distance parameter, decrease this gives more clusters.
cluster_columns = ["commandlineTokensFull", "pathScore", "accountNum", "isSystemSession"]       # extract meaningful features from process data
print("done")
print("Clustering...", end="")
(clus_events, dbcluster, x_data) = dbcluster_events(    # DBSCAN clustering groups similar processes together based on the selected features
    data = feature_procs,
    cluster_columns = cluster_columns,
    max_cluster_distance = 0.0001,      # very small distance means tight clustering
)               # similar processes (same command patterns, paths, accounts) get grouped into clusters
print("done")
print("Number of input events:", len(feature_procs))
print("Number of clustered events:", len(clus_events))
print("Merging with source data and computing rarity...", end="")

# join the clustered results back to the original process frame
procs_with_cluster = feature_procs.merge(
    clus_events[[*cluster_columns, "ClusterSize"]],
    on = ["commandlineTokensFull", "accountNum", "pathScore", "isSystemSession"],
)

# compute process pattern rarity = inverse of cluster size          # rarity calculation
procs_with_cluster["Rarity"] = 1 / procs_with_cluster["ClusterSize"]        # smaller clusters (fewer similar processes) = higher rarity scores; a process that's unique or very uncommon gets a high rarity score; common processes (large clusters) get low rarity scores
# count the number of processes for each logon ID
#lgn_proc_count = (
#    pd.concat(
#        [
#            processes_on_host.groupby("TargetLogonId")["TargetLogonId"].count(),
#            processes_on_host.groupby("SubjectLogonId")["SubjectLogonId"].count(),
#        ]
#    ).sum(level=0)      # this function was removed in pandaas >= 2.0
#).to_dict()
parts = []
if "TargetLogonId" in processes_on_host.columns:
    parts.append(processes_on_host.groupby("TargetLogonId")["TargetLogonId"].count())
if "SubjectLogonId" in processes_on_host.columns:
    parts.append(processes_on_host.groupby("SubjectLogonId")["SubjectLogonId"].count())
if parts:
    lgn_proc_count = (
        pd.concat(parts)
          .groupby(level=0)
          .sum()
          .to_dict()
    )
else:
    lgn_proc_count = {}
print("done")

# display the results
md("## **<br><hr>Sessions ordered by process rarity**")
md("Higher score indicates higher number of unusual processes")
process_rarity = (procs_with_cluster.groupby(["SubjectUserName", "SubjectLogonId"])     # groups by user and logon session, calculates mean rarity score and process count per session; sessions with high average rarity scores indicate users running many unusual processes
                  .agg({"Rarity": "mean", "TimeGenerated": "count"})
                  .rename(columns= {"TimeGenerated": "ProcessCount"})
                  .reset_index()
                  )
display(
    process_rarity
    .sort_values("Rarity", ascending=False)
    .style.bar(subset=["Rarity"], color="#d65f5f")
)

# This script helps identify:
# Anomalous user behavior: Users running rare/unusual processes
# Potential malware: Unique processes that don't cluster with normal activity
# Lateral movement: Attackers using uncommon tools or techniques
# Insider threats: Users deviating from typical process execution patterns

Input data: 22979 events
Extracting features....done
Clustering...done
Number of input events: 22979
Number of clustered events: 316
Merging with source data and computing rarity...done


## **<br><hr>Sessions ordered by process rarity**

Higher score indicates higher number of unusual processes

,SubjectUserName,SubjectLogonId,Rarity,ProcessCount
15,ian,0x5d5af2,0.607143,56
9,MSTICAdmin,0xbd57571,0.442630,38
2,MSTICAdmin,0x109c408,0.432549,10
5,MSTICAdmin,0x2e2017,0.408239,33
0,-,0x3e7,0.350000,20
10,MSTICAdmin,0xbed1e13,0.288251,21
7,MSTICAdmin,0x78225e,0.288251,21
3,MSTICAdmin,0x1e821b5,0.239992,8
8,MSTICAdmin,0xab5a5ac,0.239992,8
11,MSTICAdmin,0xc277459,0.236656,6


In [13]:
# get the logon ID of the rarest session
rarest_logon_id = process_rarity[process_rarity["Rarity"] == process_rarity.Rarity.max()].SubjectLogonId.iloc[0]

# extract processes with this logon ID
sample_processes = (
    processes_on_host
    [processes_on_host["SubjectLogonId"] == rarest_logon_id]
    [["TimeGenerated", "CommandLine"]]
    .sort_values("TimeGenerated")
)[0:40]

# compute duration of session
duration = sample_processes.TimeGenerated.max() - sample_processes.TimeGenerated.min()
md(f"**{len(sample_processes)} processes executed in {duration.total_seconds()} sec**")
display(sample_processes)
md("Note: '[PLACEHOLDER]' in the CommandLine values replaces the password value.")

**40 processes executed in 6830.497 sec**

,TimeGenerated,CommandLine
22649,2021-06-27 07:48:29.912344,"""C:\Windows\system32\cmd.exe"""
22650,2021-06-27 07:48:29.922344,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1
22653,2021-06-27 07:48:54.169344,cmd /c echo Begin Security Demo tasks
22654,2021-06-27 07:48:54.189344,cmd /c echo Any questions about the commands executed here then please contact one of
22655,2021-06-27 07:48:54.206344,cmd /c echo timb@microsoft.com; ianhelle@microsoft.com; shainw@microsoft.com
22656,2021-06-27 07:48:54.286344,ftp -s:MG06.dll
22657,2021-06-27 07:48:55.166344,cacls.exe C:\Windows\system32\cscript.exe /e /t /g SYSTEM:F
22658,2021-06-27 07:48:55.256344,net users
22659,2021-06-27 07:48:55.269344,"findstr ""abai$"""
22660,2021-06-27 07:48:55.296344,C:\Windows\system32\net1 users


Note: '[PLACEHOLDER]' in the CommandLine values replaces the password value.

# Part 3 - Detecting anomalous sequences using Markov Chain

In [14]:
# Query the data
query = """
| where TimeGenerated >= ago(60d)
| where RecordType_s == 'ExchangeAdmin'
| where UserId_s !startswith "NT AUTHORITY"
| where UserId_s !contains "prod.outlook.com"
| extend params = todynamic(strcat('{"', Operation_s, '" : ', tostring(Parameters_s), '}'))
| extend UserId = UserId_s, ClientIP = ClientIP_s, Operation = Operation_s
| project TimeGenerated=Start_Time_t, UserId, ClientIP, Operation, params
| sort by UserId asc, ClientIP asc, TimeGenerated asc
| extend begin = row_window_session(TimeGenerated, 20m, 2m, UserId != prev(UserId) or ClientIP != prev(ClientIP))
| summarize cmds=makelist(Operation), end=max(TimeGenerated), nCnds=count(), nDistinctCmds=dcount(Operation), params=makelist(params) by UserId, ClientIp, begin
| project UserId, ClientIP, nCmds, nDistinctCmds, begin, end, duration=end-begin, cmds, params
"""
exchange_df = qry_prov.Azure.OfficeActivity(add_query_items=query)
print(f"Number of events {len(exchange_df)}")
exchange_df.drop(columns="params").head()

Number of events 146


,UserId,ClientIP,nCmds,nDistinctCmds,begin,end,duration,cmds
0,NAMPRD06\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),nan,28,1,2020-06-21 02:36:46+00:00,2020-06-21 02:36:46+00:00,0 days,"[Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-Cond..."
1,NAMPRD06\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),nan,28,1,2020-06-21 05:31:34+00:00,2020-06-21 05:31:34+00:00,0 days,"[Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-Cond..."
2,NAMPRD06\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),nan,2,1,2020-06-22 02:27:06+00:00,2020-06-22 02:27:06+00:00,0 days,"[Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy]"
3,NAMPRD06\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),nan,26,1,2020-06-22 02:30:52+00:00,2020-06-22 02:30:52+00:00,0 days,"[Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-Cond..."
4,NAMPRD06\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),nan,28,1,2020-06-22 04:55:59+00:00,2020-06-22 04:55:59+00:00,0 days,"[Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-ConditionalAccessPolicy, Set-Cond..."


In [ ]:
# perform anomalous sequence analysis on the data to identify unusual patterns in Exchange server command sessions.
from msticpy.analysis.anomalous_sequence.utils.data_structures import Cmd
from msticpy.analysis.anomalous_sequence import anomalous

# data structure preparation
def process_exchange_session(session_with_params, include_vals):        # converts raw Exchange command data into structured Cmd objects with two variants
    new_ses = []
    for cmd in session_with_params:
        c = list(cmd.keys())[0]         # list command name
        par = list(cmd.values())[0]     # list command parameters
        new_pars = set()
        if include_vals:
            new_pars = dict()
        for p in par:
            if include_vals:
                new_pars[p['Name']] = p['Value']        # keep parameter values
            else:
                new_pars.add(p['Name'])                 # just keep parameter names
        new_ses.append(Cmd(name=c, params=new_pars))
    return new_ses

# session processing
sessions = exchange_df.cmds.values.tolist()     # raw command sequences
param_sessions = []         # commands with parameter names only
param_value_sessions = []   # commands with parameter names and values

for ses in exchange_df.params.values.tolist():
    new_ses_set = process_exchange_session(session_with_params=ses, include_vals=False)     # for pattern matching
    new_ses_dict = process_exchange_session(session_with_params=ses, include_vals=True)     # for detailed analysis
    param_sessions.append(new_ses_set)
    param_value_sessions.append(new_ses_dict)

data = exchange_df      # create 3 different representations of each session
data['session'] = sessions      # basic commands: just command names
data['param_session'] = param_sessions      # commands + parameters: command names with parameter names
data['param_value_session'] = param_value_sessions      # commands with parameter names and values

# anomalous sequence scoring
modelled_df = anomalous.score_sessions(
    data=data,
    session_column='param_value_session',   # use commands with parameter names and values
    window_length=3                         # analyze sequence of 3 commands
)

# visualization
anomalous.visualise_scored_sessions(
    data_with_scores=modelled_df,
    time_column='begin',                        # x-axis, timeline
    score_column='rarest_window3_likelihood',   # y-axis, anomaly score
    window_column='rarest_window3',             # represent the session in the tool-tips
    source_columns=['UserId', 'ClientIP'],      # specify any additional columns to appear in the tool-tips
)

Loading BokehJS ...

In [33]:
# exam the rare events at the bottom of the above chart
import ipywidgets as widgets

pd.set_option("display.html.table_schema", False)   # disable pandas HTML table schema for cleaner display in Jupyter notebooks.

# dynamic range calculation
likelihood_max = modelled_df["rarest_window3_likelihood"].max()
likelihood_min = modelled_df["rarest_window3_likelihood"].min()
slider_step = (likelihood_max - likelihood_min) / 20    # creates 20 steps across the full range of your data
start_val = likelihood_min + slider_step    # show the most anomalous events as the starting point

# interactive slider widget
threshold = widgets.FloatSlider(
    description = "Select likelihood threshold",
    max = likelihood_max,
    min = likelihood_min,
    value = start_val,
    step = start_val,
    layout = widgets.Layout(width = "100%"),
    style = {"description_width": "170px"},
    readout_format = ".7f"      # show 7 decimal places
)

# dynamic data filtering
def show_rows(change):
    thresh = change["new"]
    pd_disp.update(modelled_df[modelled_df["rarest_window3_likelihood"] < thresh])

threshold.observe(show_rows, names="value")
md("**Move the slider to see event sessions below the selected <i>likelihood</i> threshold**")
display(HTML("<hr>"))
display(threshold)
display(HTML("<hr>"))
md(f"Range is {likelihood_min:.7f} (min likelihood) to {likelihood_max:.7f} (max likelihood)<br><br><hr>")

# live display system
pd_disp = display(
    modelled_df[modelled_df["rarest_window3_likelihood"] < start_val],
    display_id=True
)

**Move the slider to see event sessions below the selected <i>likelihood</i> threshold**

FloatSlider(value=0.0004446243329172012, description='Select likelihood threshold', layout=Layout(width='100%'…

Range is 0.0000025 (min likelihood) to 0.0088442 (max likelihood)<br><br><hr>

,UserId,ClientIP,nCmds,nDistinctCmds,begin,end,duration,cmds,params,session,param_session,param_value_session,rarest_window3_likelihood,rarest_window3
145,timvic@contoso.onmicrosoft.com,20.185.182.48:37965,6,1,2020-07-29 20:11:27+00:00,2020-07-29 20:11:27+00:00,0 days,"[Update-RoleGroupMember, Update-RoleGroupMember, Update-RoleGroupMember, Update-RoleGroupMember,...","[{'Update-RoleGroupMember': [{'Name': 'Members', 'Value': 'CBoehmSA;pcadmin;SecurityAdmins_20075...","[Update-RoleGroupMember, Update-RoleGroupMember, Update-RoleGroupMember, Update-RoleGroupMember,...","[Cmd(name='Update-RoleGroupMember', params={'Members', 'Identity'}), Cmd(name='Update-RoleGroupM...","[Cmd(name='Update-RoleGroupMember', params={'Members': 'CBoehmSA;pcadmin;SecurityAdmins_20075581...",0.000003,"[Cmd(name='Update-RoleGroupMember', params={'Members': 'CBoehmSA;ComplianceAdmins_939735849', 'I..."


In [24]:
# print out content of the selected events/commands in more readable format
import pprint

rarest_events = (
    modelled_df[modelled_df["rarest_window3_likelihood"] < threshold.value]
    [[
        "UserId", "ClientIP", "begin", "end", "param_value_session", "rarest_window3_likelihood"
    ]]
    .rename(columns = {"rarest_window3_likelihood": "likelihood"})
    .sort_values("likelihood")
)

# structured report generation
for idx, (_, rarest_event) in enumerate(rarest_events.iterrows(), 1):
    md(f"## Event {idx}")
    display(pd.DataFrame(rarest_event[["UserId", "ClientIP", "begin", "end", "likelihood"]]))

    # detailed command analysis
    md("<hr>")
    md("**Param session details:**")
    for cmd in rarest_event.param_value_session:
        md(f"Command: {cmd.name}")
        md(pprint.pformat(cmd.params))  # ensure complex parameter dictionaries are displayed in a clean, hierarchical format that's easy to ready and analyze
    md("<hr><br>")

## Event 1

,145
UserId,timvic@contoso.onmicrosoft.com
ClientIP,20.185.182.48:37965
begin,2020-07-29 20:11:27+00:00
end,2020-07-29 20:11:27+00:00
likelihood,0.000003


<hr>

**Param session details:**

Command: Update-RoleGroupMember

...

Command: Update-RoleGroupMember

{'Identity': 'Discovery Management', 'Members': 'CBoehmSA'}

Command: Update-RoleGroupMember

{'Identity': 'Security Administrator',
 'Members': 'CBoehmSA;pcadmin;SecurityAdmins_2007558133'}

<hr><br>